In [1]:
from processorDoc import ProcesadorDocumento

file_path = "/home/sener/Descargas/RAG_functions/layout-parser-paper.pdf"
#file_path = "/home/sener/Descargas/RAG_functions/pdf_image.pdf"

#loader = PyPDFLoader(file_path)
procesador = ProcesadorDocumento(file_path, procesamiento_automatico=False)
procesador.procesarDocumento()
print(procesador.informacion)

{'producer': 'pdfTeX-1.40.21', 'creator': 'LaTeX with hyperref', 'creationdate': '2021-06-22T01:27:10+00:00', 'author': '', 'keywords': '', 'moddate': '2021-06-22T01:27:10+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/home/sener/Descargas/RAG_functions/layout-parser-paper.pdf', 'total_pages': 16, 'page': 0, 'page_label': '1'}
[2025/02/20 12:24:04] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/home/sener/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast

In [2]:
type(procesador.informacion)

list

In [3]:
from processorDoc import Chunk

chunks = Chunk(procesador.informacion)
chunksLista = chunks.chunks
print(len(chunksLista))

37


In [4]:
chunksLista[-1].metadata

{'producer': 'pdfTeX-1.40.21',
 'creator': 'LaTeX with hyperref',
 'creationdate': '2021-06-22T01:27:10+00:00',
 'author': '',
 'keywords': '',
 'moddate': '2021-06-22T01:27:10+00:00',
 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2',
 'subject': '',
 'title': '',
 'trapped': '/False',
 'source': '/home/sener/Descargas/RAG_functions/layout-parser-paper.pdf',
 'total_pages': 16,
 'page': 15,
 'page_label': '16',
 'chunk_number': 37}

In [5]:
from langchain_community.graphs.graph_document import Node, Relationship
from llm import get_embeddings, get_doc_transformer, get_chat_llm
import os

from neo4jDb import get_graph_db
graph = get_graph_db()
embedding_provider = get_embeddings()
doc_transformer = get_doc_transformer()

def process_chunk(chunk):
#for chunk in chunksLista[:40]:
    filname = os.path.basename(chunk.metadata["source"])
    chunk_id = f"{filname}.{chunk.metadata["chunk_number"]}"
    print(f"Processing - {chunk_id}")

    chunk_embedding = embedding_provider.embed_query(chunk.page_content)

    properties ={
        "filename": filname,
        "chunk_id": chunk_id,
        "text": chunk.page_content,
        "embedding": chunk_embedding
    }

    
    graph.query("""
        MERGE (d:Document {id: $filename})
        MERGE (c:Chunk {id: $chunk_id}) ON CREATE SET c.text = $text
        MERGE (d)<-[:PART_OF]-(c)
        WITH c
        CALL db.create.setNodeVectorProperty(c, "textEmbedding", $embedding)
    """, properties)
    
    graph_docs = doc_transformer.convert_to_graph_documents([chunk])
    
    for graph_doc in graph_docs:
        chunk_node = Node(id=chunk_id, type="Chunk")

        for node in graph_doc.nodes:
            graph_doc.relationships.append(
                Relationship(
                    source=chunk_node, 
                    target=node, 
                    type="HAS_ENTITY"))

    graph.add_graph_documents(graph_docs)

In [ ]:
from concurrent.futures import ThreadPoolExecutor

# Número de hilos
num_threads = 20

with ThreadPoolExecutor(max_workers=num_threads) as executor:
    executor.map(process_chunk, chunksLista)

Processing - layout-parser-paper.pdf.1
Processing - layout-parser-paper.pdf.2
Processing - layout-parser-paper.pdf.3
Processing - layout-parser-paper.pdf.4
Processing - layout-parser-paper.pdf.5
Processing - layout-parser-paper.pdf.6
Processing - layout-parser-paper.pdf.7
Processing - layout-parser-paper.pdf.8
Processing - layout-parser-paper.pdf.9
Processing - layout-parser-paper.pdf.10
Processing - layout-parser-paper.pdf.11
Processing - layout-parser-paper.pdf.12
Processing - layout-parser-paper.pdf.13
Processing - layout-parser-paper.pdf.14
Processing - layout-parser-paper.pdf.15
Processing - layout-parser-paper.pdf.16
Processing - layout-parser-paper.pdf.17
Processing - layout-parser-paper.pdf.18
Processing - layout-parser-paper.pdf.19
Processing - layout-parser-paper.pdf.20
Processing - layout-parser-paper.pdf.21
Processing - layout-parser-paper.pdf.22
Processing - layout-parser-paper.pdf.23
Processing - layout-parser-paper.pdf.24
Processing - layout-parser-paper.pdf.25
Processin

In [7]:

graph.query("""
    CREATE VECTOR INDEX `chunkVector`
    IF NOT EXISTS
    FOR (c: Chunk) ON (c.textEmbedding)
    OPTIONS {indexConfig: {
    `vector.dimensions`: 768,
    `vector.similarity_function`: 'cosine'
    }};""")

[]

In [8]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Cargar modelo de embeddings
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

# Obtener el embedding de la consulta
query_text = "¿De que trta el documento?"
query_embedding = model.encode(query_text).tolist()  # Convertir a lista para Neo4j
print(query_embedding)

[-0.004285799339413643, 0.011545456945896149, 0.009045027196407318, 0.02757652848958969, -0.05659567937254906, -0.011382430791854858, -0.04297454655170441, 0.0425054132938385, -0.0010499247582629323, 0.015608065761625767, 0.020498590543866158, -0.0008619277505204082, 0.042649537324905396, -0.03123163804411888, -0.06375948339700699, -0.04953375086188316, 0.00431950343772769, 0.018370022997260094, -0.05993765592575073, -0.009700137190520763, -0.033041104674339294, 0.04195351526141167, 0.0022905890364199877, -0.0009413186344318092, 0.015663905069231987, 0.005366922821849585, 0.03255557641386986, -0.004424667451530695, -0.04137236624956131, -0.004323347471654415, 0.04766267538070679, 0.045463282614946365, 0.006898750085383654, -0.015068561770021915, 1.5622736100340262e-06, 0.0002702425990719348, -0.0237102210521698, 0.010287545621395111, 0.021343514323234558, -0.024005185812711716, 0.007927712053060532, -0.027690110728144646, -0.005339654628187418, 0.01451863069087267, -0.03506815060973167

In [2]:
from query import QueryProcessor

processor = QueryProcessor()
    
query_text = "¿De qué trata el documento?"
context = processor.get_context(query_text)

print(context)

# Cerrar la conexión a la base de datos
processor.close()


[]


In [10]:
from neo4jDb import  get_driver
driver = get_driver()
# Consulta en Neo4j con los embeddings generados
cypher_query = """
CALL db.index.vector.queryNodes('chunkVector', 1, $embedding)
YIELD node, score
RETURN node.text, score
"""

# Ejecutar la consulta
with driver.session(database="neo4j") as session:
    result = session.run(cypher_query, embedding=query_embedding)
    for record in result:
        print(record["node.text"], record["score"])

driver.close()

segmentation. In: Proceedings of the IEEE conference on computer vision and
pattern recognition. pp. 3431–3440 (2015)
[21] Neudecker, C., Schlarb, S., Dogan, Z.M., Missier, P., Suﬁ, S., Williams, A., Wolsten-
croft, K.: An experimental workﬂow development platform for historical document
digitisation and analysis. In: Proceedings of the 2011 workshop on historical
document imaging and processing. pp. 161–168 (2011)
[22] Oliveira, S.A., Seguin, B., Kaplan, F.: dhsegment: A generic deep-learning approach
for document segmentation. In: 2018 16th International Conference on Frontiers
in Handwriting Recognition (ICFHR). pp. 7–12. IEEE (2018) 0.636751651763916


In [2]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
from langchain_neo4j import Neo4jGraph

load_dotenv(".env")

graph = Neo4jGraph(
    url=os.getenv('NEO4J_URI'),
    username=os.getenv('NEO4J_USERNAME'),
    password=os.getenv('NEO4J_PASSWORD')
)
print(graph.query('CALL db.labels()'))

[{'label': 'Chunk'}, {'label': 'Session'}]


In [5]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv(".env")

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version=os.getenv("AZURE_OPENAI_API_VERSION"), 
)

response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_MODEL"),
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Does Azure OpenAI support customer managed keys?"},
        {"role": "assistant", "content": "Yes, customer managed keys are supported by Azure OpenAI."},
        {"role": "user", "content": "Do other Azure AI services support this too?"}
    ]
)

print(response.choices[0].message.content)

Yes, many other Azure AI services support **Customer Managed Keys (CMK)** to help organizations maintain control over their data by managing encryption keys. This feature is part of Microsoft’s effort to ensure security and compliance for data-sensitive workloads. Below are examples of Azure AI services that support customer-managed keys:

### 1. **Azure Cognitive Search**
   - Azure Cognitive Search allows you to use customer-managed keys for encrypting your data at rest. This lets you maintain full control over the encryption keys stored in Azure Key Vault or Azure Managed HSM.

### 2. **Azure Machine Learning**
   - Azure Machine Learning supports customer-managed keys for encrypting both the **workspace's default storage account** and **data in Azure Machine Learning resources**, including datasets, models, and experiments.

### 3. **Azure Cognitive Services**
   - Many Cognitive Services (e.g., Text Analytics, Translator, and Form Recognizer) allow customers to use CMK to encrypt 

In [ ]:
from neo4jDb import get_driver

def get_chat_history(driver=get_driver(),session_id=None,database=None) :
    """
    Extrae el historial de conversación desde Neo4j.

    :param driver: Instancia del driver de Neo4j.
    :param session_id: ID de la sesión de chat.
    :param database: Base de datos a utilizar (por defecto "neo4j").
    :return: Lista de tuplas con (tipo de mensaje, contenido).
    """
    query = """
    MATCH (s:Session)-[:LAST_MESSAGE]->(last:Message)
    WHERE s.id = $session_id
    MATCH p = (last)<-[:NEXT*]-(msg:Message)
    UNWIND nodes(p) as msgs
    RETURN DISTINCT msgs.type, msgs.content
    """

    # Ejecutar la consulta
    with driver.session(database=database) as session:
        result = session.run(query, session_id=session_id)
        
        # Mostrar los resultados
        for record in result:
            print(f"Type: {record['msgs.type']}, Content: {record['msgs.content']}")
    driver.close()

    return None

# Ejemplo de uso:
get_chat_history(session_id="bb198b2b", database="u1234568")

Type: ai, Content: Entiendo que puede ser frustrante no obtener la información que necesitas de inmediato. Si tienes acceso a los responsables del proyecto o a los documentos oficiales, seguramente encontrarás lo que buscas. Si necesitas orientación sobre cómo buscar esa información o tienes alguna otra consulta, no dudes en pedírmelo. ¡Estoy aquí para ayudarte!
Type: human, Content: bueno :(
Type: ai, Content: Lo siento, pero no tengo acceso a información específica y detallada sobre los requisitos técnicos del Tren Maya, en particular para el tramo 5, ni sobre las cláusulas específicas que establece la Marina en relación con este proyecto. Te recomendaría consultar los documentos oficiales del proyecto, como licitaciones, especificaciones técnicas o acuerdos con la Secretaría de Marina, para obtener información precisa y actualizada. También puedes contactar directamente a las autoridades responsables del Tren Maya o a las entidades involucradas en el proyecto.
Type: human, Content: 

In [1]:
import neo4jDb
from chat import Chat

# Se crea una sesion para el chat o se pasa uno ya existente
DB_ID = "u1234568" # incializar con un minuscul

# Se inicializa el chat
chat_test = Chat(DB_ID)
chat_test.get_chat_history(session_id="ejemplo123") # Se obtiene el historial de la conversacion

Type: ai, Content: Me comentaste anteriormente que eres Alejandro y que eres becario en Sener. ¿Hay algo más que te gustaría compartir o en lo que pueda ayudarte?
Type: human, Content: quien soy?
Type: ai, Content: ¡Hasta luego, Alejandro! Mucho éxito en tu experiencia como becario en Sener. Si necesitas ayuda en el futuro, aquí estaré para apoyarte. ¡Que tengas un excelente día!
Type: human, Content: bueno adios
Type: ai, Content: ¡Claro que sí! Sener es una empresa multinacional de ingeniería y tecnología con una sólida trayectoria en diversos sectores, como energía, infraestructuras, aeroespacial y naval. En México, Sener ha participado en proyectos emblemáticos, como el Tren Maya, plantas de energía y otras iniciativas de gran impacto. 

Como becario, seguramente tendrás la oportunidad de aprender y contribuir en proyectos importantes. Si necesitas información específica sobre algún tema o proyecto dentro de Sener, no dudes en pedírmelo, y con gusto te ayudaré. ¡Bienvenido a la exp